In [ ]:
pip install sdv pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 21.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 40.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.0 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.10.0
    Uninstalling platformdirs-3.10.0:
      Successfully uninstalled platformdirs-3.10.0
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.69
    Uninstalling botocore-1.34.69:
      Successfully uninstalled botocore-1.34.69
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

: 

In [ ]:
import seaborn as sns
import pandas as pd
from ctgan import CTGAN
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load Titanic dataset
df = sns.load_dataset('titanic')

# Drop irrelevant or high-missing columns
df = df.drop(columns=['deck', 'embark_town', 'alive', 'class', 'who'])

# Drop rows with missing values
df = df.dropna()

# Convert target to binary
df['survived'] = df['survived'].astype('category')

# Identify discrete (categorical) columns
discrete_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Print preprocessed info
print("📊 Cleaned Titanic Dataset Shape:", df.shape)
print("🧩 Discrete Columns:", discrete_columns)

# Train CTGAN model
ctgan = CTGAN(epochs=100)
ctgan.fit(df, discrete_columns=discrete_columns)

# Save CTGAN model
with open("ctgan_titanic_model.pkl", "wb") as f:
    pickle.dump(ctgan, f)
print("✅ CTGAN model saved as 'ctgan_titanic_model.pkl'")

# Generate synthetic data
synthetic_data = ctgan.sample(len(df))

# Encode both datasets for ML
real_encoded = pd.get_dummies(df, drop_first=True)
synthetic_encoded = pd.get_dummies(synthetic_data, drop_first=True)

# Align both encoded dataframes
real_encoded, synthetic_encoded = real_encoded.align(synthetic_encoded, join='inner', axis=1)

# Split into features and label
X_real = real_encoded.drop('survived_1', axis=1)
y_real = real_encoded['survived_1']

X_synth = synthetic_encoded.drop('survived_1', axis=1)
y_synth = synthetic_encoded['survived_1']

# Train classifier on synthetic data, test on real data
clf = RandomForestClassifier()
clf.fit(X_synth, y_synth)
y_pred = clf.predict(X_real)

# Evaluation
print("\n🎯 Evaluation on Real Data (trained on synthetic data):\n")
print(classification_report(y_real, y_pred))


📊 Cleaned Titanic Dataset Shape: (712, 10)
🧩 Discrete Columns: ['survived', 'sex', 'embarked']


In [ ]:
import pickle

# Load the saved CTGAN model
with open("ctgan_titanic_model.pkl", "rb") as f:
    ctgan = pickle.load(f)

# Generate synthetic data (e.g., 500 rows)
synthetic_data = ctgan.sample(500)

# Display the synthetic data
print("🧪 Synthetic Titanic Data:")
print(synthetic_data)

# save to CSV
synthetic_data.to_csv("synthetic_titanic_data.csv", index=False)
print("✅ Synthetic data saved to 'synthetic_titanic_data.csv'")


🧪 Synthetic Titanic Data:
    survived  pclass     sex        age  sibsp  parch        fare embarked  \
0          1       1    male  16.031350      1      2   21.592631        S   
1          0       2  female  13.741602      0      1   -2.302188        Q   
2          0       2    male  22.798399      1      0   99.394238        S   
3          0       3    male  11.235020      1      0   25.030481        S   
4          1       2  female   7.190223      1      1   21.340302        S   
..       ...     ...     ...        ...    ...    ...         ...      ...   
495        0       2    male  17.519042      1      2   15.143950        S   
496        0       1  female   8.046962      1      1    6.965926        S   
497        0       2    male  13.977467      0      1  236.560399        C   
498        0       3    male  21.370587      0      1   72.526781        C   
499        1       2    male  20.598925      3      1   84.969059        S   

     adult_male  alone  
0          T